In [0]:
!sudo apt-get install build-essential libasound2-dev libjack-dev

!pip install magenta

# !pip install magenta-gpu

!pip install tensorflow==2.0.0-alpha0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libjack-dev is already the newest version (1:0.125.0-3).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [0]:
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow as tf

In [0]:
path_to_file = tf.keras.utils.get_file('irish.abc', 'https://raw.githubusercontent.com/aamini/introtodeeplearning_labs/2019/lab1/data/irish.abc')

# path_to_file = tf.keras.utils.get_file('irish.abc', 'http://members.yline.com/~zeiler1/abc/czdro.abc')
# path_to_file = tf.keras.utils.get_file('irish.abc', 'http://members.yline.com/~zeiler1/abc/czital.abc')



In [0]:
with open(path_to_file, 'r') as file:
    raw_data = file.read()

In [0]:
raw_songs = raw_data.split("\n\n")

In [0]:
import re

def clean_music(music_score):
    """
    :param music_score: string
    :return: cleaned_score,
    """
    p = re.compile(r"\(3(\w)\w(\w)")
    res = p.sub(r"\1\2", music_score)
    res = res.replace("!", "")
    return res


# with open("test_data", "r") as input:
#     str = input.read()
#     print(
#         clean_music(str)
#           )

In [0]:
songs_wo_tuplets = []

for song in raw_songs:
  # Remove the exclamation mark, which is not supported by the parser
  clean_song = clean_music(song)
  songs_wo_tuplets.append(clean_song)  

Refer to documentation on Magenta NoteSequence


https://nikhilsinghmus.github.io/MusicianKit/Structs/NoteSequence.html#/s:11MusicianKit12NoteSequenceV5notesSayAA0C0VGv

In [33]:
# Parse abc to notesequence

from magenta.music import abc_parser as abc_parser
import numpy as np

MAX_NOTES = 1000
NUM_PARAM = 3

# 3D numpy array: num songs x MAX_NOTES x NUM_PARAM
all_songs_data = np.empty((1, MAX_NOTES, NUM_PARAM), np.float64)

for k in range(len(songs_wo_tuplets)):
  res = abc_parser.parse_abc_tunebook(songs_wo_tuplets[k])[0]
  
  temp_data = np.empty((1, 1, NUM_PARAM), np.float64)
  notes_data = np.zeros((1, MAX_NOTES, NUM_PARAM))
  
  for key in res.keys():
    for note in res[key].notes:
      # Convert notes into a 2D numpy array
      # The 1st dimension is pitch, velocity (loudness) and interval
      # The 2nd dimension is the number of notes

      a = np.array([[[note.pitch, note.velocity, note.end_time - note.start_time]]])  
      temp_data = np.append(temp_data, a, axis=1)
      
  
  notes_data[0, :temp_data.shape[1],:temp_data.shape[2]] = temp_data
  
  all_songs_data = np.append(all_songs_data, notes_data, axis=0)
  
all_songs_data


array([[[6.7870612e-316, 9.0000000e+001, 1.0000000e+000],
        [0.0000000e+000, 0.0000000e+000, 0.0000000e+000],
        [0.0000000e+000, 0.0000000e+000, 0.0000000e+000],
        ...,
        [0.0000000e+000, 0.0000000e+000, 0.0000000e+000],
        [0.0000000e+000, 0.0000000e+000, 0.0000000e+000],
        [0.0000000e+000, 0.0000000e+000, 0.0000000e+000]],

       [[7.6000000e+001, 9.0000000e+001, 5.0000000e-001],
        [6.9000000e+001, 9.0000000e+001, 2.5000000e-001],
        [7.3000000e+001, 9.0000000e+001, 2.5000000e-001],
        ...,
        [0.0000000e+000, 0.0000000e+000, 0.0000000e+000],
        [0.0000000e+000, 0.0000000e+000, 0.0000000e+000],
        [0.0000000e+000, 0.0000000e+000, 0.0000000e+000]],

       [[7.3000000e+001, 9.0000000e+001, 2.5000000e-001],
        [6.7000000e+001, 9.0000000e+001, 2.5000000e-001],
        [6.6000000e+001, 9.0000000e+001, 2.5000000e-001],
        ...,
        [0.0000000e+000, 0.0000000e+000, 0.0000000e+000],
        [0.0000000e+000, 0.00

In [31]:
print(all_songs_data.shape)

(807, 1000, 3)


Numpy to NoteSequence reference:

https://colab.research.google.com/notebooks/magenta/hello_magenta/hello_magenta.ipynb#scrollTo=71dgCmmBli-s



In [0]:
import pickle

pickle.dump( all_songs_data, open( "all_songs_data.p", "wb" ) )

all_songs_data = pickle.load( open( "all_songs_data.p", "rb" ) )
